In [0]:
%matplotlib inline

In [3]:
import tensorflow as tf
# import cv2 as cv
import os
import numpy as np
import glob
# # from random import shuffle
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense
from tensorflow.contrib import rnn
from sklearn.utils import shuffle
import pandas as pd
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import statistics as st


Using TensorFlow backend.


In [4]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_feed_image=np.load("/content/drive/My Drive/img_emd.npy")

In [6]:
data_feed_image.shape

(301, 73, 2048)

In [7]:
df = pd.read_csv('/content/drive/My Drive/Charades_v1_train.csv')
df2 = pd.read_csv("/content/drive/My Drive/vid_ids.csv")
vids = set()
for i in df2["0"]:
  vids.add(i)
script_dict={}
for rows in df.itertuples(): 
  if rows.id in vids:
    script_dict[rows.id]=rows.script
print(script_dict)

{'46GP8': 'A person cooking on a stove while watching something out a window.', 'N11GT': 'One person opens up a folded blanket, then sneezes and leaves the stairs with a broom.', 'KRF68': 'A person runs into their laundry room. They grasp the doorknob and close the door. The person starts dressing then leaves holding their phone.', 'MJO7C': 'A person runs into their pantry holding a bottle of medicine. They grab the doorknob and close the door while watching a video on their phone.', 'S6MPZ': 'A person is eating at the desk and lying the phone down.', '7HVU8': 'A person is throwing items into a bag then starts fixing their clothes.', 'MCQO5': 'A person is undressing in the closet.  They put their clothes back into the wardrobe and drink a glass of water while looking at a picture.', 'VPIYF': 'A person is standing by a shelf and grasping his phone off of it.', 'JSUF4': 'A person is sweeping up with a broom and putting things back in their place. They take break on the sofa and start dri

In [8]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [0]:
tp=df["actions"]

In [10]:
df2.head()

,Unnamed: 0,0
0,0,MZOPX
1,1,WT9CR
2,2,BIK4X
3,3,2INEX
4,4,F76GD


In [0]:
# df2['0']
# for x in df2['0']:
#     print(x)

In [0]:
df2.rename( columns={'Unnamed: 1':'vid_id'}, inplace=True )

In [0]:
ar = []
ar = [x for x in df2['0']]
ar
#ar=['0ZOPH', 'T7449', 'BA6DD', 'OVHFT', 'SH20Q', '5L04F', 'O2VP8', 'GPR89', '9EEGQ', 'PWR6D', '0RP6B', '0OP1K']

In [0]:
df=pd.read_csv("/content/drive/My Drive/Charades_v1_train.csv")

In [15]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [0]:
ids=df["id"].values.tolist()
idx=[]
label=[]
data_feed_image_f=[]

In [61]:
len(ar)

301

In [62]:
len(idx)

0

In [63]:
data_feed_image.shape

(301, 73, 2048)

In [0]:
for i,v in enumerate(ar):
  try:
    idx.append(ids.index(v))
    data_feed_image_f.append(data_feed_image[i])
  except:
    pass

In [0]:
data_feed_image=np.asarray(data_feed_image_f)

In [66]:
data_feed_image.shape

(301, 73, 2048)

In [0]:
scripts=df['script'].values.tolist()

In [68]:
scripts[:2]

['A person cooking on a stove while watching something out a window.',
 'One person opens up a folded blanket, then sneezes and leaves the stairs with a broom.']

In [0]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [70]:
embed_fn = embed_useT('/content/drive/My Drive/USE/')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
script=[]
for _,i in enumerate(idx):
  
  script.append(scripts[i])

In [0]:
sentence_encoding=embed_fn(script)

In [0]:
data_feed_sentence=sentence_encoding

In [76]:
data_feed_sentence.shape

(301, 512)

In [0]:
vocab=set()
for i in df['script']:
  v=i.split()
  vocab.update(v)

In [0]:
vocab_len=len(vocab)

In [0]:
tok = Tokenizer(num_words=30000,lower=True)
tok.fit_on_texts(vocab)

In [0]:
seq_script=tok.texts_to_sequences(scripts)

In [0]:
max_seq_len=0
a=[]
for i in seq_script:
  a.append(len(i))
  if len(i)>max_seq_len:
    max_seq_len=len(i)

In [0]:
word2idx = tok.word_index

In [143]:
max_seq_len

84

In [0]:
mn=st.mean(a)

In [0]:
std=st.stdev(a)

In [146]:
thresh=int((mn+(3*std)))
thresh

43

In [0]:
thresh=42

In [0]:
for ind,val in enumerate(seq_script):
  seq_script[ind]=seq_script[ind][:thresh]

In [0]:
data_feed_script= pad_sequences(seq_script, maxlen=thresh+1)


In [150]:
data_feed_script.shape

(7985, 43)

In [0]:
data_feed_words=[]

In [0]:
for i in idx:
  data_feed_words.append(data_feed_script[i])

In [0]:
data_feed_words=np.asarray(data_feed_words)

In [154]:
print(data_feed_words.shape)
print(data_feed_sentence.shape)
print(data_feed_image.shape)

(301, 43)
(301, 512)
(301, 73, 2048)


In [0]:
emb={}
with open(os.path.join('/content/drive/My Drive/glove.6B.50d.txt')) as f :
  for l in f:
    v=l.split()
    w=v[0]
    emb[w]=v[1:]

In [165]:
t={}
emb_dim=50
num_words =vocab_len+1 
embedding_matrix = np.zeros((num_words,emb_dim),dtype=np.float32)
for word,i in word2idx.items():
  embedding_vector = emb.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = np.asarray(embedding_vector)
  else:
    t[word]=i
    # embedding_matrix[i] = np.asarray(emb.get("unk"))
print(t)
print(len(t))

{'selfie': 421, 'selfies': 423, "that's": 479, "they're": 481, "it's": 674, 'begans': 800, 'frig': 830, 'coffe': 864, 'sittiing': 868, "couldn't": 886, 'refridgerator': 905, "won't": 1032, "can't": 1067, "aren't": 1094, 'bathrom': 1140, "doesn't": 1147, "what's": 1151, 'doornobo': 1191, "wouldn't": 1268, "weren't": 1315, 'smiliing': 1318, 'processds': 1320, 'thens': 1358, "person's": 1378, "anyone's": 1379, "home's": 1384, "get's": 1394, "they've": 1434, 'vegins': 1454, "pantry's": 1470, "window's": 1482, 'theu': 1490, 'enterway': 1543, "don't": 1549, "night's": 1573, "bedroom's": 1577, 'begings': 1587, "didn't": 1601, "camera's": 1605, 'neaten': 1634, "who's": 1635, "clock's": 1644, 'vacum': 1668, 'quiddith': 1674, "phone's": 1779, 'visable': 1939, "house's": 1995, 'laudry': 2004, "fly's": 2190, 'bathrooom': 2201, "door's": 2244, 'decines': 2267, 'teddybear': 2286}
53


In [0]:
classes=156

In [0]:
tp=df["actions"].values

In [0]:
label=[]
for i in idx:
  label.append(int(tp[i].split()[0][1:]))

In [0]:
label=np.asarray(label)

In [206]:
label.shape

(301,)

In [0]:
label=np.reshape(label,(-1,1))

In [208]:
label.shape

(301, 1)

In [0]:
def network(text,act,sen):
  with tf.variable_scope('Action'):
    act_out=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(act)
  print(act_out.shape)

  with tf.variable_scope('Text'):
    text_out=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(text)  
  print(text_out.shape)  

  with tf.variable_scope('Sentence'):
    st=tf.keras.layers.Dense(250,activation='relu')(sen)
    st=tf.keras.layers.Dense(100,activation='relu')(st)

  fused_stacked_layer=tf.stack([act_out,text_out,st],axis=1)
  print(fused_stacked_layer.shape)

  fused_concat_layer=tf.keras.layers.Concatenate()([act_out,text_out,st])
  print(fused_concat_layer.shape)

  fused_atten_dense=tf.keras.layers.Dense(128,activation='relu')(fused_concat_layer)
  fused_atten_out=tf.keras.layers.Dense(3,activation='softmax')(fused_atten_dense)
  print(fused_atten_out.shape)

  fused_out_fin = tf.multiply(tf.expand_dims(fused_atten_out,2),fused_stacked_layer)
  print(fused_out_fin.shape)

  fused_out_fin = tf.reshape(fused_out_fin,[tf.shape(fused_out_fin)[0],fused_out_fin.shape[1]*fused_out_fin.shape[2]])
  print(fused_out_fin.shape)

  fused_fc=tf.keras.layers.Dense(170,activation='relu')(fused_out_fin)
  fused_fc_logits=tf.keras.layers.Dense(classes)(fused_fc)
  fused_fc_sft=tf.nn.softmax(fused_fc_logits)
  return fused_fc_logits,fused_fc_sft,fused_atten_out


In [0]:
test_data_feed_image=data_feed_image[-50:]
test_data_feed_words=data_feed_words[-50:]
test_data_feed_sentence=data_feed_sentence[-50:]
test_label=label[-50:]

In [0]:
label=label[:-50]
data_feed_image=data_feed_image[:-50]
data_feed_words=data_feed_words[:-50]
data_feed_sentence=data_feed_sentence[:-50]

In [363]:
max_frames =73
embedding_size = 2048
ACTION=tf.placeholder(tf.float32,[None,max_frames, embedding_size])
TEXT=tf.placeholder(tf.int32,[None,thresh+1])
SENTENCE=tf.placeholder(tf.float32,[None,512])
Y=tf.placeholder(tf.int32,[None,1])

word_embeddings=tf.Variable(embedding_matrix,name="emb",trainable=False,dtype=tf.float32)
txt= tf.nn.embedding_lookup(word_embeddings,TEXT)

logit,prob,atten=network(txt,ACTION,SENTENCE)
one_hot=tf.one_hot(indices=Y,depth=classes,axis=-1)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot,logits=logit))

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
  optimizer=tf.train.AdamOptimizer(0.001).minimize(loss)
loss_graph,x_axis,val_loss_graph=[],[],[]
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for ep in range(1000):
    l,f,_=sess.run([loss,atten,optimizer],feed_dict={ACTION:data_feed_image,TEXT:data_feed_words,Y:label,SENTENCE:data_feed_sentence})
    print(ep,"train loss :",l)
    loss_graph.append(l)
    x_axis.append(ep)
    l=sess.run(loss,feed_dict={ACTION:test_data_feed_image,TEXT:test_data_feed_words,Y:test_label,SENTENCE:test_data_feed_sentence})
    val_loss_graph.append(l)
    print("valid loss :",l)
    print("atten values  IMAGE  WORDS  SENTENCE",np.mean(f, axis=0))


  

(?, 100)
(?, 100)
(?, 3, 100)
(?, 300)
(?, 3)
(?, 3, 100)
(?, 300)
0 train loss : 5.00025
valid loss : 5.033909
atten values  IMAGE  WORDS  SENTENCE [0.3220275  0.29423636 0.38373607]
1 train loss : 4.981746
valid loss : 5.020772
atten values  IMAGE  WORDS  SENTENCE [0.308387   0.2897455  0.40186748]
2 train loss : 4.961009
valid loss : 5.011835
atten values  IMAGE  WORDS  SENTENCE [0.3164649  0.25963217 0.42390293]
3 train loss : 4.943684
valid loss : 5.000597
atten values  IMAGE  WORDS  SENTENCE [0.3330278  0.2525004  0.41447186]
4 train loss : 4.924538
valid loss : 4.9867296
atten values  IMAGE  WORDS  SENTENCE [0.3765884  0.24037544 0.38303617]
5 train loss : 4.906671
valid loss : 4.9769745
atten values  IMAGE  WORDS  SENTENCE [0.42936924 0.22700146 0.3436291 ]
6 train loss : 4.8790665
valid loss : 4.9552
atten values  IMAGE  WORDS  SENTENCE [0.5000396  0.19131394 0.30864635]
7 train loss : 4.8466277
valid loss : 4.935337
atten values  IMAGE  WORDS  SENTENCE [0.5813374  0.15800509 

KeyboardInterrupt: ignored

In [0]:
plt.plot(x_axis,loss_graph, label = "TRAIN LOSS WITH LSTMS")
# plt.plot(x_axis,val_loss_graph, label = "TRAIN LOSS WITH CNN")
plt.ylim(0,5) 
plt.xlim(0,15) 
plt.xlabel('epoch')  
plt.ylabel('loss') 
plt.title('CHECK FITTING')
plt.legend() 
plt.show() 